In [1]:
RAW_DIR = '/root/Methods/icmecheapfakes-submission/Output/Context_raw_task2'
ANNOTATION_DIR = '/root/Methods/icmecheapfakes-submission/Dataset'
OUTPUT_DIR = '/root/Methods/icmecheapfakes-submission/Output/Context_EL_task2'

In [2]:
from os.path import join, isfile
from os import listdir
import re
from itertools import groupby

import json
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [11]:
def get_image_id(filename: str) -> int:
    return int(re.search('([0-9]+)_[0-9]+\.txt', filename).group(1))

context_files = [join(RAW_DIR, file) for file in listdir(RAW_DIR) if isfile(join(RAW_DIR, file))]
context_files = sorted(context_files, key=get_image_id)

context_files_grouped_by_image_id = {key: list(val) for key, val in groupby(context_files, key = get_image_id)}
# print(len(context_files_grouped_by_image_id))
# print(context_files_grouped_by_image_id[1])

In [4]:
test_data = pd.read_json(join(ANNOTATION_DIR, 'test_data.json'), lines=True)


In [5]:
from coref import resolve_references
from entity_linking import entity_linking

def parse_entity(row, col_name):
    return entity_linking(row[col_name])

def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list

test_data['caption1_entities'] = test_data.progress_apply(lambda x: parse_entity(x, 'caption1'), axis=1)
test_data['caption2_entities'] = test_data.progress_apply(lambda x: parse_entity(x, 'caption2'), axis=1)
test_data['caption_entities'] = test_data.progress_apply(lambda x: union(x['caption1_entities'], x['caption2_entities']), axis=1)

/root/miniconda3/envs/EL/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ℹ Pipeline construct complete


Some weights of the model checkpoint at /root/.cache/refined/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /root/.cache/refined/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS e

In [6]:
test_data.head(3)

,img_local_path,caption1,caption2,context_label,article_url,maskrcnn_bboxes,caption1_modified,caption1_entities,caption2_modified,caption2_entities,bert_base_score,bert_large_score,caption_entities
0,public_test_mmsys/0.jpg,Julian Castro at his announcement in San Anton...,Julian Castro at his announcement in San Anton...,0,https://www.nytimes.com/2019/06/13/us/politics...,"[[389.9706726074219, 72.9228744506836, 505.056...","PERSON at his announcement in GPE, GPE, on DAT...","[(Julian Castro, Q970720, PERSON), (San Antoni...","PERSON at his announcement in GPE, GPE, on DATE.","[(Julian Castro, Q970720, PERSON), (San Antoni...",0.576995,0.601183,"[(Castro, Q970720, PERSON), (Tex, Q1439, GPE),..."
1,public_test_mmsys/1.jpg,Supporters of Tanzania's ruling Chama Cha Mapi...,A person sits on a truck as supporters of the ...,0,https://www.bbc.com/news/world-africa-54828934,"[[389.6280517578125, 8.949727058410645, 609.61...",Supporters of GPE's ruling ORG party come out ...,"[(Tanzania, Q924, GPE), (Chama Cha Mapinduzi, ...",A person sits on a truck as supporters of the ...,"[(Chama Cha Mapinduzi (, None, ORG), (Revoluti...",0.541939,0.729243,"[(Zanzibar, Q1774, GPE), (Friday, None, DATE),..."
2,public_test_mmsys/2.jpg,"53,000 dead people turned up on the state’s vo...",These social media posts did not link to a rec...,1,https://www.snopes.com/fact-check/53000-dead-f...,"[[0.0, 14.214579582214355, 326.70501708984375,...",CARDINAL dead people turned up on the state’s ...,"[(53,000, None, CARDINAL), (November 2018, Non...",These social media posts did not link to a rec...,"[(Florida, Q812, GPE), (November 2018, None, D...",0.234810,0.307628,"[(Florida, Q812, GPE), (53,000, None, CARDINAL..."


In [34]:
def intersect(list1, list2):
    list1 = [u[1] if u[1] else u[0] for u in list1]
    list2 = [u[1] if u[1] else u[0] for u in list2]
    # print(list1, list2)
    return len(set(list1).intersection(set(list2))) > 0

for i in tqdm(range(1000)):
    if context_files_grouped_by_image_id.get(i, None):
        # print(context_files_grouped_by_image_id[i])
        for name in context_files_grouped_by_image_id[i]:
            # print(f'Processing {name}')
            with open(name, 'r', encoding='utf8') as file:
                content = json.load(file)
        
            context = content['context']
            if context == '':
                continue

            # context_resolved = resolve_references(context)
            context_resolved = context

            new_context = []
            for sentence in context_resolved.split('\n'):
                if intersect(entity_linking(sentence + "."), test_data.iloc[i].caption_entities):
                    new_context.append(sentence)

            content['context'] = '\n'.join(new_context)
            # Change output dir
            new_filename = name.replace(RAW_DIR, OUTPUT_DIR)
            with open(new_filename, 'w+', encoding='utf8') as file:
                json.dump(content, file)


100%|██████████| 1000/1000 [43:20<00:00,  2.60s/it]  


In [32]:
print(context_files_grouped_by_image_id.get(99, None))

None
